In [1]:
import chess
import chess.engine
import random
import numpy as np

In [3]:
def action_encoder(move:chess.Move):
    # https://blog.devgenius.io/creating-an-ai-chess-engine-part-2-encoding-using-the-alphazero-method-63c3c3c3a960
    row_dict = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    try:
        from_rank, from_file, to_rank, to_file = [x for x in move.uci()]
    except ValueError:
        from_rank, from_file, to_rank, to_file, _ = [x for x in move.uci()]
    delta = (row_dict[to_rank] - row_dict[from_rank], int(to_file) - int(from_file))
    # queen moves
    directions_queenmove = (
        (+1,  0),
        (+1, +1),
        ( 0, +1),
        (-1, +1),
        (-1,  0),
        (-1, -1),
        ( 0, -1),
        (+1, -1)
    )
    # knight moves
    directions_knightmove = (
        (+2, +1),
        (+1, +2),
        (-1, +2),
        (-2, +1),
        (-2, -1),
        (-1, -2),
        (+1, -2),
        (+2, -1),
    )
    # underpromotion moves
    directions_underpromotion = (
        -1,
        0,
        1
    )
    underpromotion_type = (
        chess.KNIGHT,
        chess.ROOK,
        chess.BISHOP
    )
    # decide move types
    is_horizontal = delta[0] == 0
    is_vertical = delta[1] == 0
    is_diagonal = abs(delta[0]) == abs(delta[1])
    is_queen_promotion = move.promotion in (chess.QUEEN, None)
    is_queen_move = (
        (is_horizontal or is_vertical or is_diagonal) 
            and is_queen_promotion
    )
    is_knight_move = delta in directions_knightmove
    is_underpromotion = (
        move.promotion in underpromotion_type 
        and int(from_file) == 7 
        and int(to_file) == 8
    )
    if is_queen_move:
        direction = tuple(np.sign(delta))
        distance = np.max(np.abs(delta))
        direction_idx = directions_queenmove.index(direction)
        distance_idx = distance - 1
        move_idx = np.ravel_multi_index(
            multi_index=([direction_idx, distance_idx]),
            dims=(8,7)
        )
        action = np.ravel_multi_index(
            multi_index=((row_dict[from_rank], int(from_file)-1, move_idx)),
            dims=(8, 8, 73)
        )
        return action
    elif is_knight_move:
        direction_idx = directions_knightmove.index(delta)
        move_idx = direction_idx + 56 # knight moves start from index 56
        action = np.ravel_multi_index(
            multi_index=((row_dict[from_rank], int(from_file)-1, move_idx)),
            dims=(8, 8, 73)
        )
        return action
    elif is_underpromotion:
        delta = row_dict[to_rank] - row_dict[from_rank]
        direction_idx = directions_underpromotion.index(delta)
        promotion_idx = underpromotion_type.index(move.promotion)
        move_idx = np.ravel_multi_index(
            multi_index=([direction_idx, promotion_idx]),
            dims=(3,3)
        )
        move_idx = move_idx + 64 # underpromtion moves start from index 64
        action = np.ravel_multi_index(
            multi_index=((row_dict[from_rank], int(from_file)-1, move_idx)),
            dims=(8, 8, 73)
        )
        return action
    else:
        return None

board = chess.Board("4k3/1P6/8/8/8/8/1p6/4KN1R w K - 0 1")
for move in board.legal_moves:
    print(action_encoder(move))

4108
4107
4106
4105
4104
4103
4102
4116
2977
2978
2976
2979
2343
2350
2357
2364
1036
1090
1091
1089


In [4]:
def decode_action(board:chess.Board, action_encoded):
    num_to_file = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
    encoded_a = np.zeros([4672])
    encoded_a[action_encoded] = 1
    encoded_a = encoded_a.reshape(8,8,73)
    idx_i, idx_j, idx_k = np.where(encoded_a == 1)
    promotion = None
    for pos in zip(idx_i, idx_j, idx_k):
        i, j, k = pos
        # print(i, j, k)
    start_pos = (num_to_file[i], j+1)
    if k < 56: # queen move
        directions_queenmove = (
        (+1,  0),
        (+1, +1),
        ( 0, +1),
        (-1, +1),
        (-1,  0),
        (-1, -1),
        ( 0, -1),
        (+1, -1)
        )
        distance_idx = k%7
        direction_idx = (k-distance_idx)//7
        finish_pos = (num_to_file[i+(distance_idx+1)*directions_queenmove[direction_idx][0]], j+1+(distance_idx+1)*directions_queenmove[direction_idx][1])
        if start_pos[1] == 7 and finish_pos[1] == 8 and board.piece_at(j*8+i).symbol() in ['p', 'P']:
            promotion = 'q'
    elif k < 64: # knight move
        directions_knightmove = (
        (+2, +1),
        (+1, +2),
        (-1, +2),
        (-2, +1),
        (-2, -1),
        (-1, -2),
        (+1, -2),
        (+2, -1),
        )
        finish_pos = (num_to_file[i+directions_knightmove[k-56][0]], j+1+directions_knightmove[k-56][1])
    else: # underpromotion move
        directions_underpromotion = (
        -1,
        0,
        1
        )
        promotion_idx = (k-64)%3
        distance_idx = (k-64-promotion_idx)//3
        finish_pos = (num_to_file[i+directions_underpromotion[distance_idx]], j+2)
        if promotion_idx == 0:
            promotion = 'n'
        elif promotion_idx == 1:
            promotion = 'r'
        else:
            promotion = 'b'
    if promotion is  None:
        return str(start_pos[0])+str(start_pos[1])+str(finish_pos[0])+str(finish_pos[1])
    else:
        return str(start_pos[0])+str(start_pos[1])+str(finish_pos[0])+str(finish_pos[1])+promotion

In [5]:
for action in [4108, 4107, 4106, 4105, 4104, 4103,4116,2977,2978,2976,2979,2343,2350,2357,2364,1036,1090,1091,1089]:
    print(decode_action(board, action))

h1h8
h1h7
h1h6
h1h5
h1h4
h1h3
h1g1
f1g3
f1e3
f1h2
f1d2
e1f2
e1e2
e1d2
e1d1
b7b8q
b7b8r
b7b8b
b7b8n


In [8]:
for move in board.legal_moves:
    print(move)

h1h8
h1h7
h1h6
h1h5
h1h4
h1h3
h1h2
h1g1
f1g3
f1e3
f1h2
f1d2
e1f2
e1e2
e1d2
e1d1
b7b8q
b7b8r
b7b8b
b7b8n
